In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def stitch_images(img_list):
    # Create an ORB detector
    orb = cv2.ORB_create()

    # Create a list to hold keypoints and descriptors for each image
    keypoints_list = []
    descriptors_list = []

    # Detect keypoints and compute descriptors for each image
    for img in img_list:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = orb.detectAndCompute(gray, None)
        keypoints_list.append(keypoints)
        descriptors_list.append(descriptors)

    # Create a BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Find the homography matrices and warp the images
    base_img = img_list[0]
    for i in range(1, len(img_list)):
        matches = bf.match(descriptors_list[i-1], descriptors_list[i])
        matches = sorted(matches, key=lambda x: x.distance)

        # Extract the matched keypoints
        src_pts = np.float32([keypoints_list[i-1][m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints_list[i][m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        # Find the homography matrix
        H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

        # Warp the current image
        h, w, _ = img_list[0].shape
        img_warped = cv2.warpPerspective(img_list[i], H, (w, h))

        # Blend the images using maximum value to avoid overlaps
        base_img = np.maximum(base_img, img_warped)

    return base_img

# Load the images
img1 = cv2.imread('/content/cam1.jpeg')
img2 = cv2.imread('/content/cam2.jpeg')
img3 = cv2.imread('/content/cam3.jpeg')
img4 = cv2.imread('/content/cam4.jpeg')

# List of images
images = [img1, img2, img3, img4]

# Stitch the images
stitched_image = stitch_images(images)

# Save the result
cv2.imwrite('/mnt/data/stitched_image.jpg', stitched_image)

# Display the result
cv2_imshow(stitched_image)


error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
